# YOLO Lesion Detector

This notebook is dedicated to training and evaluating a lesion detectors on DeepLesion dataset with the following supervised model architectures for image detection with ResNet-50 backbone:
- YOLOv5 (stable, but YOLOv8 is newer),
- YOLOv11.

## Assumptions:
- Use 2D slice inputs (optionally use the neighbouring ones too),
- Resize all images to 512x512,
- Use YOLO-style Dataset format (bbox coordinates are: [x_center, y_center, w, h]).
- Use DeepLesion for training a general lesion localizer and some other like LiTS (Liver Tumor Segmentation) or CHAOS (CT liver dataset) for more specialized localizer.

## 📚 Thesis Value Summary
### Contribution and Value:
- Comparison of 1-stage vs 2-stage vs Transformer vs legacy detectors on DeepLesion	-> ✅ Fills a gap in literature
- Evaluation of improved DETRs (DINO/Deformable) -> ✅ Modern insight
- General vs specialized lesion detection -> ✅ Strong clinical relevance
- Analysis of training time, robustness, failure modes -> ✅ Engineering depth


# Google Colab only

### Download required packages

In [ ]:
!pip install -r https://raw.githubusercontent.com/pmalesa/lesion_detector/main/notebooks/requirements.txt

### Mount DeepLesion images and checkpoints from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content

# remove existing link if any
!rm -rf data/deeplesion
!rm -rf faster_rcnn_checkpoints

!mkdir -p data
!ln -s /content/drive/MyDrive/deeplesion/data/deeplesion data/deeplesion
!ls -l data

# Import all packages

In [ ]:
# General packages
from pathlib import Path

# Set paths to DeepLesion images and metadata

## Paths to unprocessed data

In [ ]:
# Google Colab
deeplesion_metadata_path = Path("data/deeplesion/deeplesion_metadata.csv")
deeplesion_image_path = Path("data/deeplesion/key_slices/")

In [ ]:
# Local
deeplesion_metadata_path = Path("../data/deeplesion_metadata.csv")
deeplesion_image_path = Path("../data/deeplesion/key_slices/")

## Paths to processed data

In [ ]:
deeplesion_data_dir = Path("data/deeplesion/")
deeplesion_preprocessed_image_path = deeplesion_data_dir / "deeplesion_preprocessed_uint8/key_slices"
deeplesion_preprocessed_metadata_path = deeplesion_data_dir / "deeplesion_preprocessed_uint8/deeplesion_metadata_preprocessed.csv"

# YOLOv5

### Download pretrained YOLOv5 model

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt
%cd ..

### Train the YOLOv5 model on the DeepLesion dataset

In [ ]:
!python yolov5/train.py --img 512 --batch 8 --epochs 100 --data data/deeplesion/deeplesion_yolo/deeplesion.yaml --weights yolov5s.pt --name deeplesion_yolov5

### Evaluate the YOLOv5 model

In [ ]:
!python yolov5/val.py --data data/deeplesion/deeplesion_yolo/deeplesion.yaml --weights yolov5/runs/train/deeplesion_yolov5/weights/best.pt --img 512 --task test

# YOLOv11